In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
from scrape_stats_function import combine_match_data

years = ["2024-2025","2023-2024","2022-2023","2021-2022","2020-2021","2019-2020","2018-2019","2017-2018","2016-2017","2015-2016","2014-2015","2013-2014","2012-2013","2011-2012","2010-2011"]
leagueinfo = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/schedule/{year}-Bundesliga-Scores-and-Fixtures"
    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/schedule/{year}-Premier-League-Scores-and-Fixtures"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/schedule/{year}-Serie-A-Scores-and-Fixtures"
    },
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/schedule/{year}-La-Liga-Scores-and-Fixtures"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/schedule/{year}-Ligue-1-Scores-and-Fixtures"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/23/{year}/schedule/{year}-Eredivisie-Scores-and-Fixtures"
    },
    {        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/32/{year}/schedule/{year}-Primeira-Liga-Scores-and-Fixtures"
    
    },
    {
        "name": "EFL Championship",
        "url": "https://fbref.com/en/comps/10/{year}/schedule/{year}-EFL-Championship-Scores-and-Fixtures"
    },
    {
        "name": "Scottish Premiership",
        "url": "https://fbref.com/en/comps/40/{year}/schedule/{year}-Scottish-Premiership-Scores-and-Fixtures"
    }
]

In [4]:
combined_list = []
combined_list = combine_match_data(years, leagueinfo, table_id=None)

200 https://fbref.com/en/comps/20/2024-2025/schedule/2024-2025-Bundesliga-Scores-and-Fixtures
200 https://fbref.com/en/comps/9/2024-2025/schedule/2024-2025-Premier-League-Scores-and-Fixtures
200 https://fbref.com/en/comps/11/2024-2025/schedule/2024-2025-Serie-A-Scores-and-Fixtures
200 https://fbref.com/en/comps/12/2024-2025/schedule/2024-2025-La-Liga-Scores-and-Fixtures
Scraping interrupted


In [5]:
df2 = pd.concat(combined_list,ignore_index=True)
df2 = df2.drop(['Notes','Match Report','Round'], axis=1)

df2.columns

int64_col = ['Wk','Attendance']
float64_col = ['xG','xG.1']


for col in int64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce').astype('Int64')
for col in float64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce').astype('float64')

df2 = df2.astype({
    'Day': 'string',
    'Home': 'string',
    'Away': 'string',
    'Venue': 'string',
    'Referee': 'string',
    'League': 'string',
    'Score': 'string',
    'Date':'string',
    'Time':'string'

})
df2.dtypes

Wk                     Int64
Day           string[python]
Date          string[python]
Time          string[python]
Home          string[python]
xG                   float64
Score         string[python]
xG.1                 float64
Away          string[python]
Attendance             Int64
Venue         string[python]
Referee       string[python]
League        string[python]
dtype: object

In [6]:
from unidecode import unidecode
normalize_name = ['Day','Home','Away','Venue','Referee','League','Score','Date','Time']

for col in normalize_name:
    df2[col] = df2[col].str.strip().str.lower()
    df2[col] = df2[col].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

In [ ]:
new_columns = {
    'xG': 'Home xG',
    'xG.1':'Away xG',
}

df2.rename(columns=new_columns, inplace=True)



np.int64(156)

In [ ]:
df2.drop_duplicates(inplace=True)

,Wk,Day,Date,Time,Home,Home xG,Score,Away xG,Away,Attendance,Venue,Referee,League
